In [42]:
# Here define data directory and path to metadata csv file
metadata_file='/nfs/zorba/DOWNLOADED/IXI/IXI.xls'
metadata_sheet='Table'
data_path='/nfs/zorba/DOWNLOADED/IXI/data/'
glob_exp='*.nii.gz'


In [43]:
# Output of this cell will be: 
# 1) scans_filtered          (list of scans that have metadata)  
# 2) df_filtered             (metadata for subjects that have available scans) and 
# 3) scanner_ids_filtered    (list of scanner ids which need to be extracted from file names)

# Import IXI spreadsheet, df_* contains the information from the demographics spreadsheet
import pandas as pd
import numpy as np
import glob
import os
workbook = pd.ExcelFile(metadata_file) 
df=workbook.parse(metadata_sheet)

#df=pd.read_excel(metadata_file,metadata_sheet)

# Replace empty cells with NaN
#df.replace('', np.nan, inplace=True)

# Generate list of relative paths to the IXI T1 scans. scans_* contains information from the downloaded files
os.chdir(data_path)
scans=sorted(glob.glob(glob_exp))

# Extract the subject ids and scanner ids from the file names
# (scanner id is not listed in the spreadsheet so need to extract from filenames in this case)
scans_scanner_ids=[] 
scans_subject_ids=[]
for s in range(0,len(scans)):
    tmp=scans[s]
    
    # get scanner id by grabbing letters in between first two dashes
    first_dash=tmp.find('-')
    second_dash=tmp.find('-',7)
    scans_scanner_ids.append(tmp[first_dash+1:second_dash])
    
    # get subject id
    scans_subject_ids.append(tmp[0:6])

# Determine the indices of the subject ids in the workbook corresponding to
# the available scans. df_subject_ids from spreadsheet, scans_subject_ids from glob
# There may be way to do this easier with pandas later
# First create list of subject_ids from the df
df_subject_ids=df['IXI_ID'].tolist()

# Turn number into 3 digit string preceded by "IXI"
df_subject_ids=['IXI' + str(int(x)).zfill(3) for x in df_subject_ids]

# Here get just the list of subjects with actual files
# First get the instersect of subject ids from metadata file and list of scans
inter = set(df_subject_ids).intersection(scans_subject_ids)

# Sort the intersection for convenience
inter=sorted(inter)

# Get lists of intersection indeces for df and scans
df_idx=[];
scans_idx=[];
for el in inter:
    df_idx.append(df_subject_ids.index(el))
    scans_idx.append(scans_subject_ids.index(el))

# Use ind to reindex the df and scans list
df_filtered=df.loc[df_idx]

#T = [L[i] for i in Idx]
scanner_ids_filtered=[scans_scanner_ids[i] for i in scans_idx]
scans_filtered=[scans[i] for i in scans_idx]
df_subject_ids_filtered=[df_subject_ids[i] for i in df_idx]

#print df_indexed['IXI_ID']
#print scans_filtered
#print scanner_ids_filtered
s=33
#print df_subject_ids_filtered[s]
#print df_filtered.get_value(s+1,'IXI_ID')
#print df_filtered.get_value(s+1,'SEX_ID (1=m, 2=f)')
#print df_filtered.get_value(s+1,'AGE')    
print list(df_filtered.index.values)[s]
print df_subject_ids_filtered[s]
print df_filtered['SEX_ID (1=m, 2=f)'][s+1]

34
IXI046
2.0


In [ ]:
# Import required libraries
# USEFUL LINKS: http://xnat.bigr.nl/index.php/Xnat:Pyxnat
#               https://wiki.xnat.org/display/XNAT16/XNAT+REST+XML+Path+Shortcuts
#               https://pythonhosted.org/pyxnat/features/files.html
#               https://wiki.mssm.edu/display/school/TMII+XNAT+FAQ+and+User+Manual

import pyxnat
import os

# Set project name
project_name="IXI"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# Create project object
project=xnat.select.project(project_name)

# First check if it already exists
if not project.exists():
    project.insert()

# Iterate over each subject in scans_filtered (or df_filtered)
# Insert T1 image file and metadata (age, sex, height, weight and scanner) 
for sub in range(295,len(df_subject_ids_filtered)): # If interating over the subject list be sure to convert incremental variable to right df_index
#for sub in range(295,296):
    #for sub in [0,1,2,3]:
    print sub
    print "Now inserting image and metadata for " + df_subject_ids_filtered[sub] + ": " + data_path + scans_filtered[sub]
    
    # Create subject object, insert subject if it doesn't already exists
    subject=project.subject(df_subject_ids_filtered[sub])
    if not subject.exists():
        subject.insert()
    
    # Create experiment object, insert experiment if it doesn't already exist
    # Session_label = MR1 prepended subject_id (If its longitudinal can do MR2, etc.)
    session_label="%s_MR1" % (df_subject_ids_filtered[sub]) 
    print session_label
    
    experiment=subject.experiment(session_label) #.create(experiments='xnat:mrSessionData')
   
    # Create experiment object, insert if it doesn't already exst
    if not experiment.exists():
        experiment.create() # By default it will make an xnat:mrSessionData experiment which is what we need
    
    ###########################################################################################
    ####### HERE INSERT METADATA
    
    # First get appropriate index of the dataframe to extract metadata for this subject
    df_index=list(df_filtered.index.values)[sub]
    
    # Now insert subject-level attributes (i.e. SEX) and experiment-level attributes (i.e. AGE)
    SEX_CODE=df_filtered.get_value(df_index,'SEX_ID (1=m, 2=f)')
    if SEX_CODE == 1:
        SEX='Male'
    elif SEX_CODE == 2:
        SEX='Female'
    else:
        print "Ooops, missing metadata info"

    print "Sex: " + SEX
    
    # Extract the subject's metadata from the df and store into variables
    # Add 1 since the dataframe index starts at 1
    DOB=str(df_filtered.get_value(df_index,'DOB'))
    AGE=str(df_filtered.get_value(df_index,'AGE'))
    STUDY_DATE=str(df_filtered.get_value(df_index,'STUDY_DATE'))
    WEIGHT=str(df_filtered.get_value(df_index,'WEIGHT') * 2.2)
    HEIGHT=str(df_filtered.get_value(df_index,'HEIGHT') * 0.394) # Convert to inches 

    print "age: " + str(AGE)
   
    # FOR FULL LIST OF ATTRIBUTES SEE https://wiki.xnat.org/display/XNAT16/XNAT+REST+XML+Path+Shortcuts
    #subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age', '10')
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender',SEX)
    try:
        subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/dob',DOB)
    except:
        print "Couldn't insert DOB"
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/weight',WEIGHT)
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/height',HEIGHT)
    
    # HERE SET ATTRIBUTES AT THE EXPERIMENT LEVEL
    # NOTE THE BELOW WILL AUTOMATICALLY SET AGE IF DOB IS ENTERED
    # Convert the scanner ID's to model name and field strength
    # Below information is obtained from: http://brain-development.org/ixi-dataset/
    SCANNER=scanner_ids_filtered[sub]
    if SCANNER=='HH':
        MODEL='Philips Medical Systems Intera'
        SITE='Hammersmith Hospital'
        FS='3T'
    elif SCANNER=='Guys':
        MODEL='Philips Medical Systems Gyroscan Intera'
        SITE="Guy's Hospital"
        FS='1.5T'
    elif SCANNER=='IOPS':
        MODEL='GE'
        SITE='Institute of Psychiatry'
        FS='1.5T'
        
    experiment.attrs.set('xnat:experimentData/date',STUDY_DATE)
    experiment.attrs.set('xnat:imageSessionData/scanner',MODEL)
    experiment.attrs.set('xnat:imageSessionData/fieldStrength',FS)
    
    # Insert it. Change NIFTI to DICOM if inserting zipped dicom
    # First standardize the orientation. Requires FSL to be installed on system
    # http://fsl.fmrib.ox.ac.uk/fsldownloads/fsldownloadmain.html
    img_reorient='tmp_reorient.nii.gz'
    cmd_reorient='fslreorient2std ' + scans_filtered[sub] + ' ' + img_reorient
    os.system(cmd_reorient)
    
    # Now insert the reoriented image
    # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
    experiment.scan('T1').resource('NIFTI').file(os.path.basename(img_reorient)).insert(
    img_reorient,
    content='T1',
    overwrite=True,
    type='T1',  # May need to change this to SPGR, MPRAGE, FLAIR etc?
    format='NIFTI')
    
    # Create ORIGINAL and SNAPSHOT to view the images in the web browser
    img_png='tmp.png'
    img_gif='tmp.gif'
    img_thumb='tmp_thumb.gif'
    cmd_create_slices='slicer ' + img_reorient + ' -S 4 1600 ' + img_png
    cmd_convert_gif='convert ' + img_png + ' ' + img_gif
    cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb
    
    os.system(cmd_create_slices)
    os.system(cmd_convert_gif)
    os.system(cmd_convert_thumb)
    
    experiment.scan('T1').resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
    img_gif,
    content='ORIGINAL',
    overwrite=True,
    format='GIF')
                                                    
    experiment.scan('T1').resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
    img_thumb,
    content='THUMBNAIL',
    overwrite=True,
    format='GIF')
                                                    

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')

#db.select.project('IXI_TEST').create()
print "ok"

295
Now inserting image and metadata for IXI341: /nfs/zorba/DOWNLOADED/IXI/data/IXI341-Guys-0906-T1.nii.gz
IXI341_MR1
Sex: Male
age: nan
Couldn't insert DOB
296
Now inserting image and metadata for IXI342: /nfs/zorba/DOWNLOADED/IXI/data/IXI342-Guys-0909-T1.nii.gz
IXI342_MR1
Sex: Male
age: 48.6789869952
297
Now inserting image and metadata for IXI344: /nfs/zorba/DOWNLOADED/IXI/data/IXI344-Guys-0905-T1.nii.gz
IXI344_MR1
Sex: Male
age: 62.0944558522
298
Now inserting image and metadata for IXI348: /nfs/zorba/DOWNLOADED/IXI/data/IXI348-Guys-0910-T1.nii.gz
IXI348_MR1
Sex: Female
age: 38.5872689938
299
Now inserting image and metadata for IXI350: /nfs/zorba/DOWNLOADED/IXI/data/IXI350-Guys-0908-T1.nii.gz
IXI350_MR1
Sex: Male
age: 62.6803559206
300
Now inserting image and metadata for IXI351: /nfs/zorba/DOWNLOADED/IXI/data/IXI351-Guys-0914-T1.nii.gz
IXI351_MR1
Sex: Female
age: 56.9199178645
301
Now inserting image and metadata for IXI353: /nfs/zorba/DOWNLOADED/IXI/data/IXI353-HH-1996-T1.nii.gz

In [ ]:
import pandas as pd
x1=pd.Index(['a','b','c','d'])
x2=pd.Index(['a','c'])
inter=pd.Index.intersection(x1,x2).tolist()
print inter
# Here retrieve original indeces
x1_idx=['a','b','c','d'].index('c')
print x1_idx